<a href="https://colab.research.google.com/github/sipocz/_dengue/blob/main/dengue_neural_02_IQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#--------------scikit import 
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

In [42]:
#_Dengue
_project="_dengue"
_PCVERSION_=False
_GITHUBVERSION_=True
_GOOGLEVERSION_=False
if _PCVERSION_:
    basedir="C:/Users/sipocz/OneDrive/Dokumentumok/GitHub"

if _GOOGLEVERSION_:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    basedir="/content/drive/My Drive/001_AI"

if _GITHUBVERSION_:
    !mkdir _dengue
    files=["dengue_features_train.csv","dengue_labels_train.csv","dengue_features_test.csv","/models/'model___79_0.04485_0.02581_.hdf5'"]
    path="https://raw.githubusercontent.com/sipocz/_dengue/main/"
    basedir="./"
    storeto=basedir+_project
    for f in files:
        fname=path+f
        !rm $f
        !wget $fname
        !mv $f $storeto

mkdir: cannot create directory ‘_dengue’: File exists
rm: cannot remove 'dengue_features_train.csv': No such file or directory
--2021-08-15 18:57:22--  https://raw.githubusercontent.com/sipocz/_dengue/main/dengue_features_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287139 (280K) [text/plain]
Saving to: ‘dengue_features_train.csv’

dengue_features_tra 100%[===================>] 280.41K  --.-KB/s    in 0.02s   

2021-08-15 18:57:22 (18.0 MB/s) - ‘dengue_features_train.csv’ saved [287139/287139]

rm: cannot remove 'dengue_labels_train.csv': No such file or directory
--2021-08-15 18:57:22--  https://raw.githubusercontent.com/sipocz/_dengue/main/dengue_labels_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.13

In [43]:

features_train=basedir+_project+"/"+"dengue_features_train.csv"
labels_train=basedir+_project+"/"+"dengue_labels_train.csv"
features_test=basedir+_project+"/"+"dengue_features_test.csv"


X_train=pd.read_csv(features_train)
y_train=pd.read_csv(labels_train)
X_test=pd.read_csv(features_test)


In [44]:
X_train["total_cases"]=y_train["total_cases"]

In [45]:
X_test.fillna(method="bfill", inplace=True)
X_train.fillna(method="bfill", inplace=True)

In [46]:

X_train_sj=X_train[X_train.city == "sj"]
X_train_iq=X_train[X_train.city == "iq"]


In [47]:

X_test_sj=X_test[X_test.city == "sj"]
X_test_iq=X_test[X_test.city == "iq"]

In [48]:
def build_training_data(dataset,labelset, history_size = 5, target_size = 23):
    '''
    source: https://www.mikulskibartosz.name/how-to-split-a-data-frame-into-time-series-for-lstm-deep-neural-network/
    '''
    start_index = history_size
    end_index = len(dataset) - target_size

    data = []
    labels = []

    for i in range(start_index, end_index):
        indices = range(i - history_size, i, 1)
        #print(indices)
        row=[]
        for j in indices:
            row=row+list(dataset.iloc[j])
        data.append(row)

    #label 
    for i in range(start_index, end_index):
        indices = range(i, i+target_size, 1)
        #print(indices)
        row=[]
        for j in indices:
            row=row+list(labelset.iloc[j])
        labels.append(row)


    data = np.array(data)
    labels=np.array(labels)
    #data = data.reshape((-1, 1))
    return data, labels


In [49]:
def datapipe_Xy(Xinp,yinp):
    
    X=Xinp.copy(deep=True)
    y=yinp.copy(deep=True)
    prediktorok=['ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw',
       'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm']
    X["reanalysis_air_temp_k"]=X["reanalysis_air_temp_k"]-273.15
    X["reanalysis_avg_temp_k"]=X["reanalysis_avg_temp_k"]-273.15
    X['reanalysis_max_air_temp_k']=X['reanalysis_max_air_temp_k']-273.15
    X["reanalysis_min_air_temp_k"]=X["reanalysis_min_air_temp_k"]-273.15
    X["reanalysis_dew_point_temp_k"]=X["reanalysis_dew_point_temp_k"]-273.15
    X["ndvi_N"]=(X["ndvi_ne"]+X["ndvi_nw"])/2
    X["ndvi_S"]=(X["ndvi_se"]+X["ndvi_sw"])/2


    Xprediktor=['ndvi_N','ndvi_S',
       'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm']
    
    yprediktor=["total_cases"]
    y['total_cases']=y['total_cases']
    
    return(X[Xprediktor],y[yprediktor])

In [50]:
#test
X2,y2=datapipe_Xy(X_train_iq,X_train_iq)

In [51]:
y2.head()

,total_cases
936,0
937,0
938,0
939,0
940,0


In [52]:
X2.head()

,ndvi_N,ndvi_S,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
936,0.162571,0.294043,25.41,23.590000,25.300000,22.034286,34.15,19.95,43.19,92.418571,25.41,16.651429,8.928571,26.400000,10.775000,32.5,20.7,3.0
937,0.246467,0.265557,60.61,23.484286,25.278571,22.208571,33.45,17.95,46.00,93.581429,60.61,16.862857,10.314286,26.900000,11.566667,34.0,20.8,55.6
938,0.174943,0.166064,55.52,23.265714,24.242857,22.472857,31.35,19.45,64.77,95.848571,55.52,17.120000,7.385714,26.800000,11.466667,33.0,20.7,38.1
939,0.186579,0.227257,5.60,22.207143,23.078571,19.647143,30.45,15.45,23.96,87.234286,5.60,14.431429,9.114286,25.766667,10.533333,31.5,14.7,30.0
940,0.325386,0.307707,62.76,23.282857,24.485714,20.807143,33.85,18.35,31.80,88.161429,62.76,15.444286,9.500000,26.600000,11.480000,33.3,19.1,4.0


In [53]:
XIQ,yIQ=build_training_data(X2,y2,9,1)

In [54]:
#XSJ.reshape(-1,9,18)

In [55]:
len(XIQ[0])/9

18.0

In [56]:
print(yIQ[1])

[1]


In [57]:
from keras.backend import clear_session
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Flatten,Bidirectional
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adam,Adamax,Nadam,Ftrl,Adadelta,Adagrad,Nadam,RMSprop
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld
#import tensorflow_addons as tfa

In [58]:
clear_session() 
dropout_1_rate=0.15
input_size=len(XIQ[0]) # timesteps
output_size=len(yIQ[0]) # features
learning_rate=0.00001

inputs=Input(shape=(input_size))
o1=Dense(500,activation="relu",kernel_initializer="HeNormal")(inputs)
d1=Dropout(dropout_1_rate,)(o1)
o2=Dense(2000,activation="relu",kernel_initializer="HeNormal")(d1)
d2=Dropout(dropout_1_rate)(o2)

o3=Dense(1000,activation="relu",kernel_initializer="HeNormal")(d2)
#d3=Dropout(dropout_1_rate)(o3)

result=Dense(output_size,activation="tanh",)(o3)
#dropout1=Dropout(rate=dropout_1_rate)(layer2)
#result=Dense(output_size,activation="sigmoid")(dropout1)

model_IQ = Model(inputs=inputs, outputs=result)
model_IQ.compile(optimizer=Adamax(learning_rate=learning_rate,), loss="mae")  #RMSprop
model_IQ.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 162)]             0         
_________________________________________________________________
dense (Dense)                (None, 500)               81500     
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              1002000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              2001000   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1001  

In [59]:

x_scaler = MinMaxScaler()
XIQ_scale = pd.DataFrame(x_scaler.fit_transform(XIQ))

y_scaler=MinMaxScaler(feature_range=(-1,1))
yIQ_scale=pd.DataFrame(y_scaler.fit_transform(yIQ))

In [60]:
yIQ_scale

,0
0,-1.000000
1,-0.982759
2,-1.000000
3,-1.000000
4,-1.000000
...,...
505,-0.896552
506,-0.913793
507,-0.862069
508,-0.982759


In [61]:
!rm model*

In [62]:
fname="./model_IQ"
callbacks = [#callback_LR,
       
        ModelCheckpoint(filepath=fname+"_{epoch}"+"_{loss:.5f}_{val_loss:.5f}_.hdf5", monitor='val_loss',
                        verbose=2, save_best_only=True, mode='min',)]

In [63]:
XIQ_scale

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161
0,0.228962,0.471363,0.120524,0.300551,0.442667,0.609431,0.521429,0.681319,0.119300,0.848334,0.120524,0.543713,0.423434,0.531915,0.525943,0.198347,0.631579,0.005522,0.431854,0.406603,0.287483,0.285452,0.440000,0.630275,0.471429,0.461538,0.127061,0.876820,0.287483,0.569033,0.535963,0.585106,0.600629,0.322314,0.642105,0.102338,0.258881,0.180413,0.263340,0.254234,...,0.608491,0.322314,0.448718,0.134180,0.774590,0.626461,0.199592,0.427260,0.552889,0.466940,0.564286,0.406593,0.046681,0.605123,0.199592,0.401711,0.810905,0.551418,0.482704,0.239669,0.525641,0.092214,0.718895,0.609295,0.233458,0.514589,0.582222,0.336750,0.721429,0.483516,0.015441,0.415279,0.233458,0.279384,0.816705,0.585106,0.773585,0.322314,0.333333,0.164182
1,0.431854,0.406603,0.287483,0.285452,0.440000,0.630275,0.471429,0.461538,0.127061,0.876820,0.287483,0.569033,0.535963,0.585106,0.600629,0.322314,0.642105,0.102338,0.258881,0.180413,0.263340,0.254234,0.311111,0.661883,0.321429,0.626374,0.178908,0.932356,0.263340,0.599829,0.298144,0.574468,0.591195,0.239669,0.631579,0.070127,0.287021,0.319530,0.026562,0.103040,...,0.482704,0.239669,0.525641,0.092214,0.718895,0.609295,0.233458,0.514589,0.582222,0.336750,0.721429,0.483516,0.015441,0.415279,0.233458,0.279384,0.816705,0.585106,0.773585,0.322314,0.333333,0.164182,0.625528,0.529092,0.254470,0.654560,0.751111,0.404750,0.750000,0.659341,0.044389,0.400861,0.254470,0.352780,0.709977,0.608156,0.666667,0.322314,0.461538,0.143567
2,0.258881,0.180413,0.263340,0.254234,0.311111,0.661883,0.321429,0.626374,0.178908,0.932356,0.263340,0.599829,0.298144,0.574468,0.591195,0.239669,0.631579,0.070127,0.287021,0.319530,0.026562,0.103040,0.166222,0.323936,0.257143,0.186813,0.066182,0.721340,0.026562,0.277844,0.438515,0.464539,0.503145,0.115702,0.000000,0.055218,0.622712,0.502428,0.297681,0.256682,...,0.773585,0.322314,0.333333,0.164182,0.625528,0.529092,0.254470,0.654560,0.751111,0.404750,0.750000,0.659341,0.044389,0.400861,0.254470,0.352780,0.709977,0.608156,0.666667,0.322314,0.461538,0.143567,0.550762,0.460061,0.109662,0.880841,0.876444,0.065437,0.914286,0.505495,0.006906,0.000000,0.109662,0.064842,0.908353,0.741135,0.726415,0.471074,0.679487,0.104730
3,0.287021,0.319530,0.026562,0.103040,0.166222,0.323936,0.257143,0.186813,0.066182,0.721340,0.026562,0.277844,0.438515,0.464539,0.503145,0.115702,0.000000,0.055218,0.622712,0.502428,0.297681,0.256682,0.341333,0.462669,0.500000,0.505495,0.087838,0.744051,0.297681,0.399145,0.469838,0.553191,0.592453,0.264463,0.463158,0.007362,0.314988,0.356441,0.077029,0.365028,...,0.666667,0.322314,0.461538,0.143567,0.550762,0.460061,0.109662,0.880841,0.876444,0.065437,0.914286,0.505495,0.006906,0.000000,0.109662,0.064842,0.908353,0.741135,0.726415,0.471074,0.679487,0.104730,0.554565,0.616811,0.164208,0.745970,0.806222,0.424739,0.764286,0.593407,0.011325,0.339831,0.164208,0.373139,0.823666,0.640957,0.714623,0.363636,0.525641,0.034787
4,0.622712,0.502428,0.297681,0.256682,0.341333,0.462669,0.500000,0.505495,0.087838,0.744051,0.297681,0.399145,0.469838,0.553191,0.592453,0.264463,0.463158,0.007362,0.314988,0.356441,0.077029,0.365028,0.422222,0.195797,0.442857,0.175824,0.002762,0.414999,0.077029,0.156886,0.816705,0.419149,0.541509,0.157025,0.242105,0.021167,0.612676,0.675712,0.423896,0.383391,...,0.726415,0.471074,0.679487,0.104730,0.554565,0.616811,0.164208,0.745970,0.806222,0.424739,0.764286,0.593407,0.011325,0.339831,0.164208,0.373139,0.823666,0.640957,0.714623,0.363636,0.525641,0.034787,0.615166,0.538803,0.462695,0.399918,0.450667,0.313173,0.728571,0.153846,0.008839,0.482573,0.462695,0.266724,0.617169,0.652482,0.694969,0.487603,0.525641,0.191791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [64]:

from keras import backend 
backend.set_value(model_IQ.optimizer.learning_rate, 0.00001) # 0.0001--

model_IQ.optimizer.iterations

<tf.Variable 'iter:0' shape=() dtype=int64, numpy=0>

In [65]:
history = model_IQ.fit(
    x=XIQ_scale,
    y=yIQ_scale,
    validation_split=0.1,
    epochs=15,
    batch_size=17,
    callbacks=callbacks,shuffle=True
   )

Epoch 1/15
27/27 [==============================] - 2s 38ms/step - loss: 0.4677 - val_loss: 0.0800

Epoch 00001: val_loss improved from inf to 0.08004, saving model to ./model_IQ_1_0.28026_0.08004_.hdf5
Epoch 2/15
27/27 [==============================] - 1s 23ms/step - loss: 0.1346 - val_loss: 0.0813

Epoch 00002: val_loss did not improve from 0.08004
Epoch 3/15
27/27 [==============================] - 1s 21ms/step - loss: 0.1279 - val_loss: 0.0820

Epoch 00003: val_loss did not improve from 0.08004
Epoch 4/15
27/27 [==============================] - 1s 22ms/step - loss: 0.1308 - val_loss: 0.0821

Epoch 00004: val_loss did not improve from 0.08004
Epoch 5/15
27/27 [==============================] - 1s 22ms/step - loss: 0.1341 - val_loss: 0.0826

Epoch 00005: val_loss did not improve from 0.08004
Epoch 6/15
27/27 [==============================] - 1s 22ms/step - loss: 0.1334 - val_loss: 0.0831

Epoch 00006: val_loss did not improve from 0.08004
Epoch 7/15
27/27 [========================

In [66]:
def grafikon(fx,ind,desc1,txt1,desc2="",txt2="",desc3="",txt3="",ngraf=2,c1='rgba(35,128,132,0.8)', c2='rgba(193,99,99,0.8)',c3='rgba(193,99,0,0.8)',title=None):
    '''
    fx: dataFrame
    ind: index
    desc1:column1
    txt1: label1
    desc2:column2
    txt2: label2
    ngraf: number of graph
    c1: color1
    c2: color2
    title: graph title
    '''
    
    #x_=[i for i in range(len(y_pred))]
    if title==None:
      title=txt1+"--"+txt2
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig0 = make_subplots(rows=1, cols=1)
    
    if True:
        fig0.add_trace(
            go.Scatter(x=fx[ind], y=fx[desc1], name=txt1, line=dict(color=c1) ,showlegend=True  ),

            row=1, col=1

        )
    
    if ngraf>1:
        fig0.add_trace(
            go.Scatter(x=fx[ind], y=fx[desc2], name=txt2, line=dict(color=c2) ,showlegend=True  ),

            row=1, col=1
        )
    if ngraf>2:
        fig0.add_trace(
            go.Scatter(x=fx[ind], y=fx[desc3], name=txt3, line=dict(color=c3) ,showlegend=True  ),

            row=1, col=1
        )



    fig0.update_layout(
        title=title,
        autosize=False,
        width=1200,
        height=600,
        
        )

    fig0.show()

In [67]:
#model_loaded=tf.keras.models.load_model('model_IQ_115_0.11411_0.05609_.hdf5',)

In [68]:
model_loaded=model_IQ

In [69]:
y_pred=model_loaded.predict(XIQ_scale)

In [70]:
y_pred[1]


array([-0.9535106], dtype=float32)

In [71]:
def backtest_train():
    y_orig=yIQ_scale
    output=pd.DataFrame(y_pred)
    output["a"]=y_orig
    output.columns=["pred","orig"]
    output["index"]=range(0,len(y_orig))
    grafikon(output,"index","orig","orig","pred","pred")

def backtest_test():
    y_orig=yIQ_scale
    output=pd.DataFrame(y_pred)
    output["a"]=y_orig
    output.columns=["pred","orig"]
    output["index"]=range(0,len(y_orig))
    grafikon(output,"index","orig","orig","pred","pred")

In [72]:
backtest_train()

In [73]:
X_a,y_a=build_training_data(X2,y2,1,1)

In [74]:
X_a[0]

array([ 0.1625714 ,  0.29404285, 25.41      , 23.59      , 25.3       ,
       22.03428571, 34.15      , 19.95      , 43.19      , 92.41857143,
       25.41      , 16.65142857,  8.92857143, 26.4       , 10.775     ,
       32.5       , 20.7       ,  3.        ])

In [83]:
clear_session() 
dropout_1_rate=0.15
input_size=len(X_a[0]) # timesteps
output_size=len(X_a[0]) # features
learning_rate=0.00001

inputs=Input(shape=(input_size))
o1=Dense(1500,activation="relu",kernel_initializer="HeNormal")(inputs)
d1=Dropout(dropout_1_rate,)(o1)
o2=Dense(9,activation="relu",kernel_initializer="HeNormal")(d1)
d2=Dropout(dropout_1_rate)(o2)

o3=Dense(1500,activation="relu",kernel_initializer="HeNormal")(d2)
#d3=Dropout(dropout_1_rate)(o3)

result=Dense(input_size,activation="tanh",)(o3)
#dropout1=Dropout(rate=dropout_1_rate)(layer2)
#result=Dense(output_size,activation="sigmoid")(dropout1)

model_a = Model(inputs=inputs, outputs=result)
model_a.compile(optimizer=Adamax(learning_rate=learning_rate,), loss="mae")  #RMSprop
model_a.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 18)]              0         
_________________________________________________________________
dense (Dense)                (None, 1500)              28500     
_________________________________________________________________
dropout (Dropout)            (None, 1500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 13509     
_________________________________________________________________
dropout_1 (Dropout)          (None, 9)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1500)              15000     
_________________________________________________________________
dense_3 (Dense)              (None, 18)                27018 

In [84]:
fname="./model_a"
callbacks = [#callback_LR,
       
        ModelCheckpoint(filepath=fname+"_{epoch}"+"_{loss:.5f}_{val_loss:.5f}_.hdf5", monitor='val_loss',
                        verbose=2, save_best_only=True, mode='min',)]

In [85]:

from keras import backend 
backend.set_value(model_a.optimizer.learning_rate, 0.00001) # 0.0001--

model_a.optimizer.iterations

<tf.Variable 'iter:0' shape=() dtype=int64, numpy=0>

In [86]:
x_a_scaler = MinMaxScaler(feature_range=(-1,1))
Xa_scale = pd.DataFrame(x_a_scaler.fit_transform(X_a))



In [87]:
Xa_scale

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-0.542077,-0.057274,-0.758953,-0.398898,-0.114667,0.218862,0.042857,0.362637,-0.761401,0.696669,-0.758953,0.087425,-0.153132,0.063830,5.188679e-02,-0.603306,0.263158,-0.988956
1,-0.136292,-0.186795,-0.425034,-0.429096,-0.120000,0.260550,-0.057143,-0.076923,-0.745877,0.753639,-0.425034,0.138067,0.071926,0.170213,2.012579e-01,-0.355372,0.284211,-0.795325
2,-0.482239,-0.639174,-0.473320,-0.491532,-0.377778,0.323766,-0.357143,0.252747,-0.642184,0.864712,-0.473320,0.199658,-0.403712,0.148936,1.823899e-01,-0.520661,0.263158,-0.859746
3,-0.425959,-0.360939,-0.946877,-0.793920,-0.667556,-0.352127,-0.485714,-0.626374,-0.867635,0.442679,-0.946877,-0.444311,-0.122970,-0.070922,6.289308e-03,-0.768595,-1.000000,-0.889564
4,0.245424,0.004855,-0.404639,-0.486635,-0.317333,-0.074663,0.000000,0.010989,-0.824324,0.488102,-0.404639,-0.201711,-0.060325,0.106383,1.849057e-01,-0.471074,-0.073684,-0.985275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,-0.537136,-0.760705,-0.436513,-0.245052,-0.242667,0.590637,-0.142857,0.560440,-0.517720,0.942959,-0.436513,0.504192,-0.598608,0.276596,-1.886792e-02,-0.404959,0.368421,-0.882201
514,-0.033176,-0.251230,-0.989091,-0.139359,-0.050667,-0.249274,0.242857,-0.296703,-0.951385,0.039194,-0.989091,-0.330026,0.222738,-0.099291,-2.452830e-01,-0.355372,0.115789,-0.990797
515,0.271733,-0.146457,-0.475407,0.342175,0.463111,0.611481,0.385714,0.670330,-0.751402,0.517707,-0.475407,0.526775,-0.011601,0.539007,2.704403e-01,-0.123967,0.621053,-0.900607
516,-0.553236,-0.571946,-0.179718,0.055295,0.120000,0.522296,0.214286,0.098901,0.144104,0.656565,-0.179718,0.427203,-0.389791,0.283688,-2.220446e-16,-0.239669,0.473684,-0.865268


In [ ]:
history = model_a.fit(
    x=Xa_scale,
    y=Xa_scale,
    validation_split=0.1,
    epochs=1500,
    batch_size=1,
    callbacks=callbacks,shuffle=True
   )

Epoch 1/1500
466/466 [==============================] - 2s 2ms/step - loss: 0.4998 - val_loss: 0.3943

Epoch 00001: val_loss improved from inf to 0.39427, saving model to ./model_a_1_0.46547_0.39427_.hdf5
Epoch 2/1500
466/466 [==============================] - 1s 1ms/step - loss: 0.3878 - val_loss: 0.3201

Epoch 00002: val_loss improved from 0.39427 to 0.32005, saving model to ./model_a_2_0.37283_0.32005_.hdf5
Epoch 3/1500
466/466 [==============================] - 1s 1ms/step - loss: 0.3424 - val_loss: 0.2769

Epoch 00003: val_loss improved from 0.32005 to 0.27686, saving model to ./model_a_3_0.32962_0.27686_.hdf5
Epoch 4/1500
466/466 [==============================] - 1s 1ms/step - loss: 0.3111 - val_loss: 0.2534

Epoch 00004: val_loss improved from 0.27686 to 0.25336, saving model to ./model_a_4_0.30388_0.25336_.hdf5
Epoch 5/1500
466/466 [==============================] - 1s 1ms/step - loss: 0.2813 - val_loss: 0.2428

Epoch 00005: val_loss improved from 0.25336 to 0.24285, saving mo